In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
cd My\ Drive

/gdrive/My Drive


In [ ]:
pip install transformers

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import csv
import os
import string
import six
import time
import torch.nn as nn
import torch.nn.functional as F
from transformers import *

import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# Read data from csv file
def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")

def preprocess(text:str):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text


class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label


class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example


class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir):
    """Gets a collection of `InputExample`s for prediction."""
    raise NotImplementedError()

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.io.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
      lines = []
      for line in reader:
        lines.append(line)
      return lines

class ZQAProcessor(DataProcessor):
  """Processor for the Zalo Quation Answering data set."""

  def _read_csv(self, input_file, islabel=True):
    """Reads a tab separated value file."""
    print('read_csv..........')
    df = pd.read_csv(input_file,lineterminator='\n', usecols=['question', 'text', 'label']) if islabel else pd.read_csv(input_file, usecols=['question', 'text', '__id__', 'id'])
    lines = []
    for i, r in df.iterrows():
      line = [r['question'], r['text'], str(r['label'])] if islabel else [r['question'], r['text'], r['__id__']+'-'+ r['id']]
      lines.append(line)
    return lines

  def get_train_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_csv(os.path.join(data_dir, "train.csv")), "train")

  def get_dev_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_csv(os.path.join(data_dir, "val.csv")), "val")

  def get_test_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_csv(os.path.join(data_dir, "test.csv"), islabel=False), "test")

  def get_labels(self):
    """See base class."""
    return ["False", "True"]

  def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
      guid = "%s-%s" % (set_type, i)
      text_a = preprocess(convert_to_unicode(line[0]))
      text_b = preprocess(convert_to_unicode(line[1]))
      if set_type == "test":
        guid = line[2]
        label = "False"
      else:
        label = convert_to_unicode(line[2])
      examples.append(
          InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples


In [ ]:
processor = ZQAProcessor()
examples = processor.get_train_examples("Dataset")
label_list = processor.get_labels()

read_csv..........


In [ ]:
train_examples, valid_examples = train_test_split(examples, test_size=0.15, random_state=42, shuffle=True)

In [ ]:
# Custom model
class PairSequenceModel(BertPreTrainedModel):
    def __init__(self, model_name:str, config):
        super(PairSequenceModel, self).__init__(config)
        self.bert_model = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 2)

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        outputs = self.bert_model(input_ids=b_input_ids, attention_mask=b_input_mask, token_type_ids=b_token_ids)
        pooled_output = self.dropout(outputs[1])
        logits = self.classifier(pooled_output)
        loss_function = nn.CrossEntropyLoss()
        loss = loss_function(logits, labels)
        return (loss, logits)


In [ ]:
# Load pre-trained model
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

config = BertConfig.from_pretrained("bert-base-multilingual-cased")

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")


In [ ]:
# # Load pre-trained bert-base-multilingual-cased fine tune on Vietnamese
# tf_checkpoint_path="./Model/bert4vn/Squad2_b24_model.ckpt-16289"
# bert_config_file="Model/bert4vn/bert_config.json"
# pytorch_dump_path="Model/bert4vn.pt"
# config = BertConfig.from_json_file(bert_config_file)
# print("Building PyTorch model from configuration: {}".format(str(config)))
# # model = BertForSequenceClassification(config)

# #     # Load weights from tf checkpoint
# # load_tf_weights_in_bert(model, config, tf_checkpoint_path)

In [ ]:
def prepare_data(examples, bs, max_length):
    input_ids = []
    token_ids = []
    attention_masks =[]
    labels = []

    for example in examples:
        encoding = tokenizer(example.text_a, example.text_b, return_tensors="np", max_length=max_length, truncation=True, padding='max_length')
        input_ids.append(encoding["input_ids"].reshape((max_length, 1)))
        token_ids.append(encoding["token_type_ids"].reshape((max_length, 1)))
        attention_masks.append(encoding["attention_mask"].reshape((max_length, 1)))
        if example.label == "True":
            labels.append([[1]])
        elif example.label == "False":
            labels.append([[0]])
    # convert to torch tensor
    tensor_input_ids = torch.squeeze(torch.tensor(input_ids))
    tensor_attention_masks = torch.squeeze(torch.tensor(attention_masks))
    tensor_token_ids = torch.squeeze(torch.tensor(token_ids))
    tensor_labels = torch.tensor(labels).squeeze()

    
    data = TensorDataset(tensor_input_ids, tensor_token_ids, tensor_attention_masks, tensor_labels)
    data_sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=data_sampler, batch_size=bs)
    return dataloader

train_dataloader = prepare_data(train_examples, bs=16, max_length=512)
valid_dataloader = prepare_data(valid_examples, bs=16, max_length=512)

In [ ]:
# Set parameters to update
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=1e-5,
    eps=1e-8
)

In [ ]:
# Learning scheduler
from transformers import get_linear_schedule_with_warmup

epochs = 5
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm, trange
import numpy as np
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
model.to(device)

PairSequenceModel(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values, testing_loss_values = [], [], []
best_score = 0
num_epoch_no_improve = 0
for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0
    t0 = time.time()
    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_token_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.

        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(input_ids=b_input_ids, attention_mask=b_input_mask, token_type_ids=b_token_ids, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print(" \nTime per epoch: ", (time.time()-t0))
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_token_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(input_ids=b_input_ids, attention_mask=b_input_mask, token_type_ids=b_token_ids, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').squeeze().numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend(np.argmax(logits, axis=1))
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    valid_f1_score = f1_score(true_labels, predictions)
    if best_score < valid_f1_score:
        best_score = valid_f1_score
    else: 
        num_epoch_no_improve += 1
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(accuracy_score(true_labels, predictions)))
    print("Validation F1-Score: {}".format(valid_f1_score))
    print()

    # if num_epoch_no_improve == 2:
    #     print('Early stopping !')
    #     torch.save(model.state_dict(), 'Model/model_new_nn.pt')
    #     break


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

 
Time per epoch:  840.4444608688354
Average train loss: 0.4104462401194037


Epoch:  20%|██        | 1/5 [14:48<59:15, 888.98s/it]

Validation loss: 0.3665916655212641
Validation Accuracy: 0.8461538461538461
Validation F1-Score: 0.7490996398559423

 
Time per epoch:  840.6194796562195
Average train loss: 0.2689423441495785


Epoch:  40%|████      | 2/5 [29:38<44:27, 889.04s/it]

...

Validation loss: 0.46680687684017946
Validation Accuracy: 0.8645564961354435
Validation F1-Score: 0.7842907385697537

 
Time per epoch:  840.0264644622803
Average train loss: 0.14228797629388465


Epoch:  80%|████████  | 4/5 [59:16<14:48, 888.96s/it]

Validation loss: 0.5978101268479162
Validation Accuracy: 0.8663967611336032
Validation F1-Score: 0.790052053209948

 
Time per epoch:  839.5462906360626
Average train loss: 0.11589804030577275


Epoch: 100%|██████████| 5/5 [1:14:04<00:00, 888.84s/it]

Validation loss: 0.6618839522049872
Validation Accuracy: 0.8645564961354435
Validation F1-Score: 0.7842907385697537



In [ ]:
torch.save(model.state_dict(), 'Model/model_bert_custom.pt')

In [ ]:
for b in valid_dataloader:
    batch = b
    break

batch = tuple(t.to(device) for t in batch)
b_input_ids, b_token_ids, b_input_mask, b_labels = batch

In [ ]:
out = model(input_ids=b_input_ids, attention_mask=b_input_mask, token_type_ids=b_token_ids, labels=b_labels)
out

(tensor(0.8586, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor([[ 7.6080e-01, -7.4042e-02],
         [ 1.7373e+00, -1.0946e+00],
         [-5.4649e-01,  1.2954e+00],
         [ 1.4350e-01,  5.9250e-01],
         [ 9.7047e-01, -2.4431e-01],
         [ 1.6930e+00, -1.0541e+00],
         [-1.9336e-01,  9.1567e-01],
         [ 2.0352e+00, -1.4772e+00],
         [ 6.2579e-01,  1.2519e-01],
         [ 1.2092e+00, -5.0172e-01],
         [ 8.0745e-01, -6.6168e-02],
         [ 1.2728e+00, -5.8226e-01],
         [ 1.7370e+00, -1.1165e+00],
         [-4.9766e-01,  1.2726e+00],
         [ 9.1971e-01, -2.2640e-01],
         [ 7.2588e-01, -5.9270e-04]], device='cuda:0', grad_fn=<AddmmBackward>))

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 3            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   15304 MB |   15304 MB |   32946 MB |   17642 MB |\n|       from large pool |   15300 MB |   15300 MB |   32940 MB |   17640 MB |\n|       from small pool |       3 MB |       3 MB |       6 MB |       2 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   15304 MB |   15304 MB |   32946 MB |   17642 MB |\n|       from large pool |   15300 MB |   15300 MB |